使用线性规划和整数规划方法来解决这个问题，可以通过建立一个目标函数和约束条件来优化生产过程中相关的成本、次品率等因素。以下是使用线性规划和整数规划解决这个问题的步骤和思路：

### 1. 决策变量定义
为了构建数学模型，首先需要定义决策变量：
- 设 \(x_i\) 为是否对零配件 \(i\) 进行检测的二进制变量，\(x_i = 1\) 表示检测，\(x_i = 0\) 表示不检测。
- 设 \(y_j\) 为生产中半成品 \(j\) 是否合格的二进制变量，\(y_j = 1\) 表示合格，\(y_j = 0\) 表示不合格。
- 设 \(z\) 为成品是否合格的二进制变量，\(z = 1\) 表示合格成品，\(z = 0\) 表示次品。
  
这些变量有助于描述生产过程中不同阶段的状态。

### 2. 目标函数
目标是最小化生产过程中发生的总成本。总成本包括以下几个部分：
- **采购成本**：各个零配件的采购单价。
- **检测成本**：如果对零配件或半成品进行检测，则产生检测成本。
- **装配成本**：装配零配件、半成品、成品所需的费用。
- **次品损失**：次品的拆解费用和市场损失。

因此，目标函数可以表示为：
\[
\text{Minimize } Z = \sum_{i=1}^{8} \text{采购成本}(i) + \sum_{i=1}^{8} x_i \cdot \text{检测成本}(i) + \sum_{j=1}^{3} y_j \cdot \text{装配成本}(j) + z \cdot \text{市场损失}
\]
其中，\(x_i\)、\(y_j\)、\(z\) 都是二进制变量，表示是否进行检测、是否合格等决策。

### 3. 约束条件
约束条件用于限制决策变量的取值，并确保生产过程满足要求。根据图1和表2中的数据，可以定义以下几类约束条件：

#### (1) 次品率约束
次品率与是否进行检测密切相关。假设不进行检测时，零配件 \(i\) 的次品率为 \(p_i\)，如果进行检测则次品率为 \(p_i(1 - \text{检测有效率})\)。因此可以写出次品率约束：
\[
y_j = (1 - p_j) \cdot \prod_{i \in S_j} (1 - p_i(1 - x_i))
\]
其中，\(S_j\) 是组成半成品 \(j\) 的零配件集合。

#### (2) 装配和生产约束
成品由3个半成品组装而成，只有所有半成品都合格时，成品才合格。因此可以写出成品的装配约束：
\[
z = y_1 \cdot y_2 \cdot y_3
\]
这表示如果所有半成品 \(y_1, y_2, y_3\) 都合格，则成品 \(z\) 合格。

#### (3) 成品的市场价值和次品损失
根据表2中的信息，如果成品不合格，则会产生市场损失和次品拆解费用。我们需要根据成品是否合格来决定是否产生这些损失：
\[
z = 1 \Rightarrow \text{利润} = \text{市场价值} - Z
\]
\[
z = 0 \Rightarrow \text{损失} = \text{市场损失} + \text{拆解费用}
\]

### 4. 完整模型
综上，整个问题的数学模型可以表述为：

\[
\text{Minimize } Z = \sum_{i=1}^{8} \text{采购成本}(i) + \sum_{i=1}^{8} x_i \cdot \text{检测成本}(i) + \sum_{j=1}^{3} y_j \cdot \text{装配成本}(j) + z \cdot \text{市场损失}
\]
\[
\text{subject to:}
\]
1. \(y_j = (1 - p_j) \cdot \prod_{i \in S_j} (1 - p_i(1 - x_i)), \quad j = 1, 2, 3\)
2. \(z = y_1 \cdot y_2 \cdot y_3\)
3. \(x_i \in \{0, 1\}, y_j \in \{0, 1\}, z \in \{0, 1\}\)


通过这些工具，最终可以找到一个优化的生产决策方案，最小化总成本或最大化利润。

In [ ]:
import pulp
import itertools

# 创建线性规划问题，目标是最小化成本
problem = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)

# 定义决策变量 x_i: 是否检测第i个零配件，1表示检测，0表示不检测
x = pulp.LpVariable.dicts("x", range(1, 9), lowBound=0, upBound=1, cat='Binary')

# 定义决策变量 y_j: 半成品j是否合格，1表示合格，0表示不合格
y = pulp.LpVariable.dicts("y", range(1, 4), lowBound=0, upBound=1, cat='Binary')

# 定义决策变量 z_P: 成品是否合格，1表示合格，0表示不合格
z_P = pulp.LpVariable("z_P", lowBound=0, upBound=1, cat='Binary')

# 定义零配件的次品率
defective_rate = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

# 定义采购单价、检测成本
purchase_costs = [2, 8, 12, 2, 8, 12, 8, 12]
inspection_costs = [1, 1, 2, 1, 1, 2, 1, 2]

# 定义其他成本
assembly_cost = 8  # 半成品和成品的装配成本
assembly_inspection_cost = 4  # 半成品检测成本
disassembly_semi_cost = 6  # 半成品的拆解成本
disassembly_final_cost = 10  # 成品的拆解成本
market_loss = 40  # 市场损失

# 1. 定义目标函数 (采购成本 + 检测成本 + 装配成本 + 拆解成本 + 市场损失)
# 采购成本
C_purchase = pulp.lpSum([purchase_costs[i] for i in range(8)])

# 装配成本
C_assembly = 3 * assembly_cost + assembly_cost

# 半成品检测成本
C_assembly_inspection = 3 * assembly_inspection_cost

# 遍历所有可能的检测组合 (0表示不检测，1表示检测)
for combination in itertools.product([0, 1], repeat=8):
    # 为每个决策变量赋值组合中的检测状态
    for i in range(1, 9):
        x[i] = combination[i-1]

    # 定义检测成本
    C_inspect = pulp.lpSum([inspection_costs[i] * combination[i] for i in range(8)])

    # 拆解成本
    C_disassembly_semi = disassembly_semi_cost * pulp.lpSum([1 - y[j+1] for j in range(3)])
    C_disassembly_final = disassembly_final_cost * (1 - z_P)

    # 市场损失
    C_market = market_loss * (1 - z_P)

    # 更新目标函数：最小化总成本
    problem.setObjective(C_purchase + C_inspect + C_assembly + C_assembly_inspection + C_disassembly_semi + C_disassembly_final + C_market)

    # 重新添加约束条件
    problem += y[1] <= 1 - defective_rate[0] * (1 - x[1]) - defective_rate[1] * (1 - x[2]) - defective_rate[2] * (1 - x[3])
    problem += y[2] <= 1 - defective_rate[3] * (1 - x[4]) - defective_rate[4] * (1 - x[5]) - defective_rate[5] * (1 - x[6])
    problem += y[3] <= 1 - defective_rate[6] * (1 - x[7]) - defective_rate[7] * (1 - x[8])

    problem += z_P <= y[1]
    problem += z_P <= y[2]
    problem += z_P <= y[3]

    # 求解问题
    problem.solve()

    # 打印当前组合及其结果
    print("Current combination:", combination)
    print("Status:", pulp.LpStatus[problem.status])

    # 打印总成本
    print("Total Cost:", pulp.value(problem.objective))

    # 打印各零配件的检测决定
    for i in range(1, 9):
        print(f"Inspect part {i}: {combination[i-1]}")

    # 打印半成品的合格情况
    for j in range(1, 4):
        print(f"Semi-finished product {j} is qualified: {int(pulp.value(y[j]))}")

    # 打印成品的合格情况
    print(f"Final product is qualified: {int(pulp.value(z_P))}")
    print("-" * 50)  # 分隔线


Current combination: (0, 0, 0, 0, 0, 0, 0, 0)
Status: Optimal
Total Cost: 176.0
Inspect part 1: 0
Inspect part 2: 0
Inspect part 3: 0
Inspect part 4: 0
Inspect part 5: 0
Inspect part 6: 0
Inspect part 7: 0
Inspect part 8: 0
Semi-finished product 1 is qualified: 0
Semi-finished product 2 is qualified: 0
Semi-finished product 3 is qualified: 0
Final product is qualified: 0
--------------------------------------------------
Current combination: (0, 0, 0, 0, 0, 0, 0, 1)
Status: Optimal
Total Cost: 178.0
Inspect part 1: 0
Inspect part 2: 0
Inspect part 3: 0
Inspect part 4: 0
Inspect part 5: 0
Inspect part 6: 0
Inspect part 7: 0
Inspect part 8: 1
Semi-finished product 1 is qualified: 0
Semi-finished product 2 is qualified: 0
Semi-finished product 3 is qualified: 0
Final product is qualified: 0
--------------------------------------------------
Current combination: (0, 0, 0, 0, 0, 0, 1, 0)
Status: Optimal
Total Cost: 177.0
Inspect part 1: 0
Inspect part 2: 0
Inspect part 3: 0
Inspect part 4: 